In [1]:
import pandas as pd
from pytesseract import pytesseract
import cv2
from typing import List, Tuple
import matplotlib.pyplot as plt

import os
import sys
import glob

pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

import warnings
warnings.simplefilter("ignore")

In [2]:
sys.path.append("../scripts")
from load_data import Loader
from utils import *
from logo_extraction import *

loader = Loader()

In [8]:
DIRECTORY_PATH = "/home/amanuel_zewdu/creative_image_optimization/data/Challenge_Data/Assets/"

IMAGES_EXT = ["JPG","PNG","GIF","WEBP","TIFF","PSD","RAW","BMP","HEIF","INDD","JPEG"]

VIDEO_EXT = ["WEBM","MPG","MP2","MPEG","MPE","MPV","OGG","MP4","M4P","M4V","AVI","WMV","MOV","QT","FLV","SWF"]

### identify assets with logo information

In [3]:
bucket = "s3://10ac-batch-6/data/w11/Challenge_Data.zip"
file_path = "Challenge_Data/performance_data.csv"

df = loader.load_csv(bucket,file_path)

In [4]:
logos = df.copy()
logos.drop(columns=["preview_link","ER","CTR"],inplace=True)

In [5]:
logos["all_files"] = logos.game_id.apply(lambda x:get_files_name(f'{DIRECTORY_PATH}{x}/'))

In [12]:
logos.head()

,game_id,all_files
1,bfbf7a2b7ac635e67877b1ab87fd6629,"[endtext.png, landing_2.jpg, video.mp4, end-ha..."
2,945010afbf9a55bbdffcb0895f946155,"[landing_2.jpg, video.mp4, end-mtlaurel.png, e..."
3,e68e20f592457b875ce29757ab855dfe,"[endtext.png, thumbnail.jpg, landing_2.jpg, en..."
4,adunit-nestle-purina-friskies-mob,"[logo.png, game_2.png, cta.png, engagement_ani..."
5,adunit-lionsgate-uwomt-user-slider-sensory-vid...,"[logo.png, cta.png, shade.png, engagement_anim..."


In [6]:
logos['concat'] = logos.all_files.apply(lambda x: " ".join(x))

In [14]:
logos.concat.str.lower().str.contains("logo").value_counts()

False    474
True     431
Name: concat, dtype: int64

only 431 creative assets directory contains information about logo

In [7]:
contain_logo = logos[logos.concat.str.lower().str.contains("logo")]

In [16]:
contain_logo.sample(5)

,game_id,all_files,concat
831,9931c8470e7412c45c890f4f7c59c1f4,"[logo.png, video.mp4, cta.png, engageable-laye...",logo.png video.mp4 cta.png engageable-layer.pn...
181,42c5b4b367187c29ac8fdd78c70837d6,"[logo.png, arrowdown.png, star24.png, star33e....",logo.png arrowdown.png star24.png star33e.png ...
684,06615e719bb053415093a4572a1f1d31,"[end.jpg, logo.png, video.mp4, landing_endfrma...",end.jpg logo.png video.mp4 landing_endfrmae.pn...
88,adunit-mouser-user-choice-flight4-canada-cpe-a...,"[option2-image.png, logo.png, arrow1.png, back...",option2-image.png logo.png arrow1.png backgrou...
863,c8fc9c5e6ec515c610c73f772ed5d3a4,"[endtext.png, logo.png, video.mp4, end-mtlaure...",endtext.png logo.png video.mp4 end-mtlaurel.pn...


In [17]:
not_contain_logo = logos[~logos.concat.str.lower().str.contains("logo")]

In [18]:
not_contain_logo.sample(5)

,game_id,all_files,concat
15,7fc571f85358c5d37efafde99b6896d7,"[thumbnail.jpg, landing_2.jpg, video.mp4, MPU-...",thumbnail.jpg landing_2.jpg video.mp4 MPU-clic...
261,267aa12df20c85a8c215e3d6f7a74879,"[video.mp4, rev-end-bg.jpg, fs-engage.png, cta...",video.mp4 rev-end-bg.jpg fs-engage.png cta.png...
782,f52568446cd557df708c44efb668789c,"[endtext.png, video.mp4, video-cta.jpg, engage...",endtext.png video.mp4 video-cta.jpg engagement...
625,623735c2b4b2d78c61e08073dbaa352d,"[thumbnail.jpg, landing_2.jpg, video.mp4, MPU-...",thumbnail.jpg landing_2.jpg video.mp4 MPU-clic...
729,adunit-gm-ces-user-choice-mpu,"[endframe_4.png, bg.png, arrow-3.png, cta.png,...",endframe_4.png bg.png arrow-3.png cta.png endt...


### work with the first asset group

In [19]:
# img_path = "/home/amanuel_zewdu/creative_image_optimization/data/Challenge_Data/Assets/2524439faeafa1c2ca1a27cac00a97b1/_preview.png"
# logo_path = "/home/amanuel_zewdu/creative_image_optimization/data/Challenge_Data/Assets/2524439faeafa1c2ca1a27cac00a97b1/logo.png"

img_path = "/home/amanuel_zewdu/creative_image_optimization/data/Challenge_Data/Assets/79c36d2adb94900291f5ddf1f6580c43/endframe_2.png"
logo_path = "/home/amanuel_zewdu/creative_image_optimization/data/Challenge_Data/Assets/79c36d2adb94900291f5ddf1f6580c43/logo.png"

In [8]:
contain_logo['file_name'] = contain_logo.all_files.apply(lambda x: filter_list(x,"logo"))

In [9]:
contain_logo.drop(columns=["all_files","concat"],inplace=True)

In [22]:
contain_logo.sample(5)

,game_id,file_name
749,e17b6db0e4fb777ab569ef0a81b5000e,[logo.png]
807,9e068e03269af51054a95146bdaf3d45,[logo.png]
207,2b7e702f208b7fd60d15d0bdadd269f4,"[logo.png, LOGO.png, logo01.png]"
124,2a7069f1ccdc33dd1f7a200e3026b4ad,[logo.png]
295,adunit-chevy-user-slider-seat-interior-v1-cpe-...,[logo.png]


In [10]:
back_up_logo_list = contain_logo.copy()

In [24]:
contain_logo = back_up_logo_list.copy()

In [25]:

contain_logo[["shape","top_left","bottom_right"]] =  contain_logo.apply(lambda x: pd.Series(find_logo_position(x.game_id,x.file_name)) ,axis = 1)

In [26]:
contain_logo.sample(5)

,game_id,file_name,shape,top_left,bottom_right
717,adunit-nestle-maggi-varieties-bouillon-en-uae-mpu,[logo.png],"(604, 452)","(65, 374)","(175, 439)"
135,adunit-mouser-user-choice-v3-us-cpe-av-mob,[logo.png],"(652, 1046)","(533, 482)","(1037, 648)"
303,e0c8ecc4ba1bc299fb4e43f6faccffb9,[logo.png],"(900, 600)","(0, 0)","(600, 900)"
636,6de62384e53319712bf7910e4db4e189,[logo.png],"(900, 600)","(365, 525)","(599, 615)"
256,edd62a365ff46ae8c2e7a88dcb48ba56,[logo.png],"(900, 600)","(54, 411)","(544, 489)"


In [27]:
# Logo Area Ratio
contain_logo[["LAR"]] =  contain_logo.apply(lambda x: pd.Series(find_logo_area_ratio(x["shape"],x["top_left"],x["bottom_right"])) ,axis = 1)
contain_logo["LAR"] =  contain_logo.LAR.apply(lambda x: x if x <=1 else 0 )

In [28]:
contain_logo.describe()

,LAR
count,431.000000
mean,0.132191
std,0.230897
min,0.000000
25%,0.026848
50%,0.050883
75%,0.122676
max,1.000000


In [29]:
# contain_logo.sample(5)

In [30]:
contain_logo[contain_logo['LAR'] == 0 ].shape

(63, 6)

In [ ]:
contain_logo.to_csv('game_id_with_logo_positions_and_ratio.csv')